In [ ]:
# @title 1: Cargar Librerías Necesarias para Carga y Visualización

import pandas as pd
import numpy as np
import pickle # Para cargar archivos .pkl
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML
import plotly.io as pio

# Configurar Plotly para Colab (por si acaso)
pio.renderers.default = 'colab'

print("Librerías cargadas.")

Librerías cargadas.


In [121]:
# @title 2: Cargar Datos desde Archivos Pickle

import os

print("--- Cargando datos desde archivos .pkl ---")
print("Asegúrate de haber subido los archivos .pkl al panel 'Archivos' de Colab.")

# --- Nombres de archivo esperados ---
files_to_load = {
    'df_palabras': "palabras_reducidas_export.pkl",
    'df_segmentos': "segmentos_reducidos_export.pkl",
    'df_textos': "textos_reducidos_export.pkl",
    'dict_cot_segmentos': "analisis_cot_detallado_export.pkl",
    'dict_json_palabras': "analisis_json_palabras_export.pkl",
    'dict_cot_global': "analisis_cot_global_export.pkl"
}

# --- Diccionario para guardar los datos cargados ---
loaded_data = {}
all_files_found = True

# --- Bucle de Carga ---
for data_key, filename in files_to_load.items():
    print(f"\nIntentando cargar: {filename} (como '{data_key}')...")
    if os.path.exists(filename):
        try:
            with open(filename, 'rb') as f:
                loaded_object = pickle.load(f)
            loaded_data[data_key] = loaded_object
            print(f"  -> ¡Éxito! Cargado en loaded_data['{data_key}']")
            # Pequeña verificación del tipo cargado
            if isinstance(loaded_object, pd.DataFrame):
                print(f"     (Tipo: DataFrame, Filas: {len(loaded_object)})")
            elif isinstance(loaded_object, dict):
                 print(f"     (Tipo: Diccionario, Claves: {len(loaded_object.keys())})")
            else:
                 print(f"     (Tipo: {type(loaded_object)})")
        except Exception as e:
            print(f"  ⛔ ERROR al cargar o leer el archivo pickle: {e}")
            all_files_found = False # Marcar que algo falló
    else:
        print(f"  (!) Archivo '{filename}' NO encontrado en el directorio actual.")
        all_files_found = False # Marcar que faltan archivos

# --- Resumen y Asignación a Variables Globales (opcional) ---
print("\n--- Resumen de Carga ---")
if not loaded_data:
    print("(!) No se cargó ningún dato.")
elif not all_files_found:
    print("(!) Advertencia: Faltaron algunos archivos o hubo errores durante la carga.")
else:
    print("✅ Todos los archivos .pkl esperados fueron encontrados y cargados.")

# Opcional: Asignar a variables globales con nombres más cortos para facilidad de uso
# (Asegúrate de que las claves en loaded_data coincidan)
df_palabras_viz = loaded_data.get('df_palabras')
df_segmentos_viz = loaded_data.get('df_segmentos')
df_textos_viz = loaded_data.get('df_textos')
dict_cot_segmentos_viz = loaded_data.get('dict_cot_segmentos')
dict_json_palabras_viz = loaded_data.get('dict_json_palabras')
dict_cot_global_viz = loaded_data.get('dict_cot_global')

# Verificar que las variables principales (DataFrames) existen
print("\nVerificando variables principales:")
if df_palabras_viz is not None: print("- df_palabras_viz: OK")
else: print("- df_palabras_viz: NO CARGADO")
if df_segmentos_viz is not None: print("- df_segmentos_viz: OK")
else: print("- df_segmentos_viz: NO CARGADO")
if df_textos_viz is not None: print("- df_textos_viz: OK")
else: print("- df_textos_viz: NO CARGADO")


print("\n--- Fin de Carga de Datos ---")

--- Cargando datos desde archivos .pkl ---
Asegúrate de haber subido los archivos .pkl al panel 'Archivos' de Colab.

Intentando cargar: palabras_reducidas_export.pkl (como 'df_palabras')...
  -> ¡Éxito! Cargado en loaded_data['df_palabras']
     (Tipo: DataFrame, Filas: 990)

Intentando cargar: segmentos_reducidos_export.pkl (como 'df_segmentos')...
  -> ¡Éxito! Cargado en loaded_data['df_segmentos']
     (Tipo: DataFrame, Filas: 98)

Intentando cargar: textos_reducidos_export.pkl (como 'df_textos')...
  -> ¡Éxito! Cargado en loaded_data['df_textos']
     (Tipo: DataFrame, Filas: 19)

Intentando cargar: analisis_cot_detallado_export.pkl (como 'dict_cot_segmentos')...
  -> ¡Éxito! Cargado en loaded_data['dict_cot_segmentos']
     (Tipo: Diccionario, Claves: 19)

Intentando cargar: analisis_json_palabras_export.pkl (como 'dict_json_palabras')...
  -> ¡Éxito! Cargado en loaded_data['dict_json_palabras']
     (Tipo: Diccionario, Claves: 19)

Intentando cargar: analisis_cot_global_export.p

In [122]:
# @title 3: Verificación Detallada de Datos Cargados

import pandas as pd
import numpy as np
from IPython.display import display, Markdown # Usar Markdown para títulos

print("--- Celda 3 (Revisado): Verificación Detallada de Datos ---")

# --- Función Auxiliar para Chequear Columnas y Nulos ---
def check_dataframe(df, df_name, expected_cols, check_null_cols):
    """Realiza verificaciones comunes en un DataFrame y devuelve True si es usable."""
    display(Markdown(f"### Verificando: {df_name}"))
    if df is None or not isinstance(df, pd.DataFrame):
        print("(!) Error: DataFrame no cargado o no es un DataFrame válido.")
        return False
    if df.empty:
        print("(!) Advertencia: DataFrame está vacío.")
        return False # No se puede usar si está vacío

    print(f"-> Encontrado ({len(df)} filas).")
    available_columns = df.columns.tolist()

    # Verificar columnas clave
    print("\n1. Verificación de Columnas Clave:")
    missing_expected = [col for col in expected_cols if col not in available_columns]
    if missing_expected:
        print(f"   (!) FALTAN columnas clave necesarias: {missing_expected}")
        # Decidir si es crítico. Si faltan coordenadas, es crítico.
        if any(c.endswith(('_x', '_y', '_z')) for c in missing_expected):
             print("   ⛔ ERROR CRÍTICO: Faltan columnas de coordenadas. No se puede visualizar.")
             return False
    else:
        print("   ✅ Todas las columnas clave esperadas están presentes.")

    # Analizar nulos en columnas seleccionadas
    print("\n2. Análisis de Nulos en Columnas Importantes:")
    cols_to_check = [c for c in check_null_cols if c in available_columns]
    if not cols_to_check:
         print("   (No hay columnas específicas para verificar nulos)")
    else:
        for col in cols_to_check:
            num_nan = df[col].isnull().sum()
            if num_nan == 0:
                print(f"   - '{col}': OK (0 nulos)")
            else:
                percent_nan = (num_nan / len(df)) * 100
                print(f"   - '{col}': {num_nan} nulos ({percent_nan:.1f}%)")
                if col.endswith(('_x','_y','_z')):
                     print("     (!) Advertencia: Hay nulos en coordenadas, esas filas no se graficarán.")
                elif col == 'embedding':
                     print("     (!) Advertencia: Hay nulos en embeddings, reducción/ploteo fallará para esas filas.")


    # Mostrar info general y ejemplo
    print("\n3. Información General y Ejemplo:")
    df.info(verbose=False) # Info concisa
    cols_to_show_sample = [c for c in expected_cols if c in available_columns][:8] # Mostrar hasta 8 columnas clave
    if not cols_to_show_sample and len(available_columns) > 0: # Fallback si no hay esperadas
         cols_to_show_sample = available_columns[:5]
    if cols_to_show_sample:
        print("\n   Ejemplo (primeras filas):")
        display(df[cols_to_show_sample].head(3))
    else:
        print("   (No hay columnas adecuadas para mostrar ejemplo).")

    return True # Si llegó hasta aquí, es usable

# --- Ejecutar Verificaciones ---

# Verificar Palabras
expected_cols_palabras = ['texto_id', 'segmento_idx', 'palabra_idx_in_segment', 'palabra_texto', 'categoria', 'lemma', 'genero', 'numero', 'embedding', 'pca_2d_x', 'umap_2d_x']
check_null_cols_palabras = ['embedding', 'categoria', 'lemma', 'pca_2d_x', 'umap_2d_x']
palabras_ok = check_dataframe(globals().get('df_palabras_viz'), "DataFrame de Palabras", expected_cols_palabras, check_null_cols_palabras)

# Verificar Segmentos
expected_cols_segmentos = ['texto_id', 'segmento_idx', 'embedding', 'analisis_json_completo', 'pca_2d_x', 'umap_2d_x']
check_null_cols_segmentos = ['embedding', 'analisis_json_completo', 'pca_2d_x', 'umap_2d_x']
segmentos_ok = check_dataframe(globals().get('df_segmentos_viz'), "DataFrame de Segmentos", expected_cols_segmentos, check_null_cols_segmentos)

# Verificar Textos
expected_cols_textos = ['texto_id', 'embedding', 'pca_2d_x', 'umap_2d_x']
check_null_cols_textos = ['embedding', 'pca_2d_x', 'umap_2d_x']
textos_ok = check_dataframe(globals().get('df_textos_viz'), "DataFrame de Textos Completos", expected_cols_textos, check_null_cols_textos)

# --- Resumen Final de Verificación ---
print("\n\n--- Resumen Final de Verificación de DataFrames ---")
if palabras_ok: print("✅ DataFrame de Palabras: Listo para usar.")
else: print("❌ DataFrame de Palabras: Problemas detectados o no cargado.")
if segmentos_ok: print("✅ DataFrame de Segmentos: Listo para usar.")
else: print("❌ DataFrame de Segmentos: Problemas detectados o no cargado.")
if textos_ok: print("✅ DataFrame de Textos Completos: Listo para usar.")
else: print("❌ DataFrame de Textos Completos: Problemas detectados o no cargado.")

print("\n--- Fin de Celda 3 (Verificación Detallada) ---")

--- Celda 3 (Revisado): Verificación Detallada de Datos ---


### Verificando: DataFrame de Palabras

-> Encontrado (990 filas).

1. Verificación de Columnas Clave:
   ✅ Todas las columnas clave esperadas están presentes.

2. Análisis de Nulos en Columnas Importantes:
   - 'embedding': OK (0 nulos)
   - 'categoria': OK (0 nulos)
   - 'lemma': 612 nulos (61.8%)
   - 'pca_2d_x': OK (0 nulos)
   - 'umap_2d_x': OK (0 nulos)

3. Información General y Ejemplo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Columns: 51 entries, texto_id to umap_3d_z
dtypes: float32(10), float64(5), int64(2), object(34)
memory usage: 355.9+ KB

   Ejemplo (primeras filas):


,texto_id,segmento_idx,palabra_idx_in_segment,palabra_texto,categoria,lemma,genero,numero
0,Texto_1,0,0,Me,pronombre,NaN,None,singular
1,Texto_1,0,1,gustaría,verbo,NaN,NaN,singular
2,Texto_1,0,2,cortarle,verbo,NaN,NaN,None


### Verificando: DataFrame de Segmentos

...

,texto_id,segmento_idx,embedding,analisis_json_completo,pca_2d_x,umap_2d_x
0,Texto_1,0,"[-0.060000945, 0.056589235, -0.02383715, -0.01...","[{'categoria': 'pronombre', 'palabra_analizada...",0.377948,-0.550188
1,Texto_2,0,"[0.007489225, 0.026745263, 0.008314279, 0.0113...","[{'categoria': 'verbo', 'palabra_analizada': '...",0.415345,-0.379248
2,Texto_3,0,"[0.00034429508, 0.017901659, -0.008097587, 0.0...","[{'categoria': 'pronombre', 'palabra_analizada...",0.365220,-0.553485


### Verificando: DataFrame de Textos Completos

-> Encontrado (19 filas).

1. Verificación de Columnas Clave:
   ✅ Todas las columnas clave esperadas están presentes.

2. Análisis de Nulos en Columnas Importantes:
   - 'embedding': OK (0 nulos)
   - 'pca_2d_x': OK (0 nulos)
   - 'umap_2d_x': OK (0 nulos)

3. Información General y Ejemplo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Columns: 18 entries, texto_id to umap_3d_z
dtypes: float32(10), float64(5), object(3)
memory usage: 2.1+ KB

   Ejemplo (primeras filas):


,texto_id,embedding,pca_2d_x,umap_2d_x
0,Texto_1,"[-0.037022393, 0.057320707, -0.027191967, 0.01...",0.167254,6.937783
1,Texto_2,"[0.011024177, 0.038843084, -0.0059259715, 0.01...",0.158290,5.364787
2,Texto_3,"[0.0021676808, 0.032636456, 0.0039843484, 0.03...",0.098213,5.839240




--- Resumen Final de Verificación de DataFrames ---
✅ DataFrame de Palabras: Listo para usar.
✅ DataFrame de Segmentos: Listo para usar.
✅ DataFrame de Textos Completos: Listo para usar.

--- Fin de Celda 3 (Verificación Detallada) ---


In [125]:
# @title 4: Visualización 2D/3D de Textos con Conteos

import plotly.express as px
from IPython.display import display, HTML, clear_output
import pandas as pd
import numpy as np
import textwrap

print("--- Celda 4: Visualización 2D/3D Textos con Conteos ---")

# --- Verificar Datos ---
variables_ok_viz_text = False # Reset flag
df_textos_viz = globals().get('df_textos_viz')
df_segmentos_viz = globals().get('df_segmentos_viz') # Necesitamos segmentos para contar
df_palabras_viz = globals().get('df_palabras_viz')   # Necesitamos palabras para contar

# Verificar existencia y tipo
if df_textos_viz is None or not isinstance(df_textos_viz, pd.DataFrame) or df_textos_viz.empty:
    print("(!) Error: DataFrame 'df_textos_viz' no encontrado o vacío.")
elif df_segmentos_viz is None or not isinstance(df_segmentos_viz, pd.DataFrame): # Permitir vacío
    print("(!) Advertencia: DataFrame 'df_segmentos_viz' no encontrado. No se mostrará conteo de segmentos.")
elif df_palabras_viz is None or not isinstance(df_palabras_viz, pd.DataFrame): # Permitir vacío
    print("(!) Advertencia: DataFrame 'df_palabras_viz' no encontrado. No se mostrará conteo de palabras.")
else:
    print(f"DataFrame de Textos encontrado ({len(df_textos_viz)} filas).")
    variables_ok_viz_text = True # OK para proceder


# --- Calcular y Fusionar Conteos (Si los datos base están OK) ---
if variables_ok_viz_text:
    print("\nCalculando y fusionando conteos de segmentos y palabras...")

    # --- ASEGURAR QUE LAS COLUMNAS NO EXISTAN ANTES DEL MERGE ---
    cols_to_drop = ['segment_count', 'word_count', 'texto_snippet', 'hover_text_display']
    for col in cols_to_drop:
        if col in df_textos_viz.columns:
            print(f"  -> Eliminando columna existente '{col}' antes de recalcular.")
            df_textos_viz = df_textos_viz.drop(columns=col)
    # ----------------------------------------------------------

    # Conteo de Segmentos
    if df_segmentos_viz is not None and not df_segmentos_viz.empty and 'texto_id' in df_segmentos_viz.columns:
        segment_counts = df_segmentos_viz.groupby('texto_id').size().reset_index(name='segment_count')
        # Ahora el merge no debería encontrar conflictos
        df_textos_viz = pd.merge(df_textos_viz, segment_counts, on='texto_id', how='left')
        df_textos_viz['segment_count'] = df_textos_viz['segment_count'].fillna(0).astype(int)
        print("  -> Conteo de segmentos añadido.")
    else:
        df_textos_viz['segment_count'] = 0
        print("  (!) Conteo de segmentos no calculado.")

    # Conteo de Palabras
    if df_palabras_viz is not None and not df_palabras_viz.empty and 'texto_id' in df_palabras_viz.columns:
        word_counts = df_palabras_viz.groupby('texto_id').size().reset_index(name='word_count')
        # Ahora el merge no debería encontrar conflictos
        df_textos_viz = pd.merge(df_textos_viz, word_counts, on='texto_id', how='left')
        df_textos_viz['word_count'] = df_textos_viz['word_count'].fillna(0).astype(int)
        print("  -> Conteo de palabras añadido.")
    else:
        df_textos_viz['word_count'] = 0
        print("  (!) Conteo de palabras no calculado.")

    # Preparar Hover Text (Ahora siempre se recreará después de eliminar)
    dict_cot_global_viz = globals().get('dict_cot_global_viz')
    if dict_cot_global_viz:
         def get_text_info_v4(text_id):
             data = dict_cot_global_viz.get(text_id, {}); original_text = data.get('texto_original', '')
             snippet = textwrap.shorten(original_text, width=200, placeholder="..."); length = len(original_text)
             return pd.Series([snippet, length])
         # Aplicar para crear/recrear columnas
         df_textos_viz[['texto_snippet', 'texto_length']] = df_textos_viz['texto_id'].apply(get_text_info_v4)

         def format_hover_text_v4(row):
             if not isinstance(row['texto_snippet'], str): return "N/A"
             clean_text = ' '.join(row['texto_snippet'].split()); wrapped = textwrap.fill(clean_text, width=80); return wrapped.replace('\n', '<br>')
         # Crear/recrear columna hover
         df_textos_viz['hover_text_display'] = df_textos_viz.apply(format_hover_text_v4, axis=1)
         print("  -> Snippet/Longitud/Hover creados/actualizados.")
    else:
         print("  (!) Hover será básico (falta dict CoT Global).")
         # Asegurar columnas fallback si no se crearon antes
         if 'hover_text_display' not in df_textos_viz.columns: df_textos_viz['hover_text_display'] = df_textos_viz['texto_id']
         if 'texto_length' not in df_textos_viz.columns: df_textos_viz['texto_length'] = 100
         if 'segment_count' not in df_textos_viz.columns: df_textos_viz['segment_count'] = 0 # Asegurar existencia
         if 'word_count' not in df_textos_viz.columns: df_textos_viz['word_count'] = 0 # Asegurar existencia



# --- Parámetros de Visualización ---
print("\n--- Configuración ---")
# @markdown Selecciona método y dimensiones:
param_metodo_text_23d = "PCA" # @param ["UMAP", "PCA", "t-SNE"]
param_dimensiones_text_23d = 2 # @param [2, 3] ## <-- REINTRODUCIDO ##
# @markdown ---
# @markdown **Apariencia:**
# @markdown Tamaño base de los puntos:
param_tamano_base_text_23d = 12 # @param {type:"slider", min:5, max:25, step:1}
# @markdown Hacer tamaño proporcional a la longitud del texto?
param_tamano_variable_23d = True # @param {type:"boolean"}
# @markdown Colorear cada punto con un color distinto?
param_colorear_text_23d = True # @param {type:"boolean"}
# @markdown Opacidad de los puntos:
param_opacidad_text_23d = 0.75 # @param {type:"slider", min:0.1, max:1.0, step:0.1}
# @markdown Mostrar ejes X e Y (o X/Y/Z)?
param_mostrar_ejes_23d = True # @param {type:"boolean"}
# @markdown Plantilla de estilo de Plotly:
param_template_23d = "seaborn" # @param ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white"]


# --- Generación y Visualización 2D/3D ---
clear_output(wait=True)
print(f"--- Generando Gráfico {param_dimensiones_text_23d}D Mejorado de Textos Completos ---")

if not variables_ok_viz_text:
    print("(!) No se puede generar gráfico, faltan datos.")
# Verificar columnas necesarias para hover y tamaño ahora que se añadieron
elif not all(c in df_textos_viz.columns for c in ['hover_text_display', 'texto_length', 'segment_count', 'word_count']):
     print("(!) Error: Faltan columnas calculadas ('hover_text_display', 'texto_length', 'segment_count', 'word_count'). Revisa la preparación.")
else:
    method = param_metodo_text_23d
    dimensions = param_dimensiones_text_23d
    base_marker_size = param_tamano_base_text_23d
    size_col = 'texto_length' if param_tamano_variable_23d else None
    color_col = 'texto_id' if param_colorear_text_23d else None
    opacity = param_opacidad_text_23d
    show_axes = param_mostrar_ejes_23d
    template = param_template_23d
    df_plot_text = df_textos_viz

    # Determinar y verificar columnas de coordenadas
    x_col = f'{method.lower().replace("-","")}_{dimensions}d_x'
    y_col = f'{method.lower().replace("-","")}_{dimensions}d_y'
    z_col = f'{method.lower().replace("-","")}_{dimensions}d_z' if dimensions == 3 else None
    coord_cols = [x_col, y_col] + ([z_col] if z_col else [])
    required_cols = ['texto_id', 'hover_text_display', 'texto_length', 'segment_count', 'word_count'] + coord_cols

    missing_cols = [col for col in required_cols if col not in df_plot_text.columns]
    if missing_cols:
        print(f"⛔ ERROR: Columnas requeridas no encontradas para '{method}' {dimensions}D: {', '.join(missing_cols)}")
    else:
        df_plot_valid_text = df_plot_text.dropna(subset=coord_cols).copy()
        # Asegurar que columna de tamaño sea numérica si se usa
        if size_col and not pd.api.types.is_numeric_dtype(df_plot_valid_text[size_col]):
             df_plot_valid_text[size_col] = pd.to_numeric(df_plot_valid_text[size_col], errors='coerce')
             df_plot_valid_text = df_plot_valid_text.dropna(subset=[size_col])

        if df_plot_valid_text.empty:
            print("⛔ ERROR: No hay datos válidos después de filtrar NaNs.")
        else:
            print(f"-> Graficando {len(df_plot_valid_text)} textos completos.")
            title = f'Mapa Semántico {dimensions}D de Textos Completos ({method})'

            # Preparar Hover CON CONTEOS
            custom_data_cols = ['texto_id', 'hover_text_display', 'texto_length', 'segment_count', 'word_count']
            custom_hovertemplate = (
                f"<b>Texto ID:</b> %{{customdata[0]}}<br>"
                f"<b>Longitud:</b> %{{customdata[2]}} chars<br>"
                f"<b>Segmentos:</b> %{{customdata[3]}}<br>" # <-- Nuevo
                f"<b>Palabras:</b> %{{customdata[4]}}<br>"   # <-- Nuevo
                f"<br>"
                f"<b>Inicio del Texto:</b><br>%{{customdata[1]}}<br>"
                f"<extra></extra>"
            )

            # Preparar mapa de colores si aplica
            color_discrete_map = None
            unique_texts = sorted(df_plot_valid_text['texto_id'].unique())
            if color_col:
                colors = px.colors.qualitative.Plotly; color_discrete_map = {text: colors[i % len(colors)] for i, text in enumerate(unique_texts)}

            try:
                # --- Usar función correcta (2D o 3D) ---
                plot_func = px.scatter if dimensions == 2 else px.scatter_3d
                plot_args = {'data_frame': df_plot_valid_text, 'x': x_col, 'y': y_col, 'title': title,
                             'text': 'texto_id', 'color': color_col, 'color_discrete_map': color_discrete_map,
                             'size': size_col, 'custom_data': custom_data_cols, 'template': template }
                if dimensions == 3: plot_args['z'] = z_col # Añadir Z para 3D
                if size_col: plot_args['size_max'] = base_marker_size * 3

                fig_text_23d = plot_func(**plot_args)
                # ---------------------------------------

                # Actualizar trazas
                marker_dict_update = {'opacity': opacity, 'line': dict(width=1, color='DarkSlateGrey')}
                if not size_col: marker_dict_update['size'] = base_marker_size
                fig_text_23d.update_traces(marker=marker_dict_update, textposition='top center',
                                           textfont_size=9, hovertemplate=custom_hovertemplate)

                # Actualizar layout según dimensiones
                layout_config = {'hovermode': 'closest', 'margin': dict(l=20, r=20, t=60, b=20),
                                 'showlegend': bool(color_col) and len(unique_texts) <= 15,
                                 'legend_title_text':'Texto ID' if color_col else None}
                if dimensions == 2:
                     layout_config['xaxis'] = dict(visible=show_axes, showticklabels=show_axes, title=f"{method} Dim 1" if show_axes else None, zeroline=False, showgrid=False)
                     layout_config['yaxis'] = dict(visible=show_axes, showticklabels=show_axes, title=f"{method} Dim 2" if show_axes else None, zeroline=False, showgrid=False)
                else: # 3D
                     layout_config['scene'] = dict(xaxis=dict(visible=show_axes, showticklabels=show_axes, title=f"{method} Dim 1" if show_axes else ''),
                                                   yaxis=dict(visible=show_axes, showticklabels=show_axes, title=f"{method} Dim 2" if show_axes else ''),
                                                   zaxis=dict(visible=show_axes, showticklabels=show_axes, title=f"{method} Dim 3" if show_axes else ''))
                     # Ocultar ejes completamente si show_axes es False
                     if not show_axes:
                          layout_config['scene'].update(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

                fig_text_23d.update_layout(**layout_config)

                # Mostrar como HTML
                print(f"-> Gráfico {dimensions}D Mejorado generado. Mostrando HTML...")
                html_text_23d = fig_text_23d.to_html(full_html=False, include_plotlyjs='cdn')
                display(HTML(html_text_23d))
                print("(Pasa el ratón sobre los puntos para ver detalles)")

            except Exception as e:
                print(f"⛔ ERROR al generar/mostrar el gráfico {dimensions}D: {e}"); import traceback; traceback.print_exc()

print(f"\n--- Fin de Celda 4 - {param_dimensiones_text_23d}D con Conteos) ---")

--- Generando Gráfico 2D Mejorado de Textos Completos ---
-> Graficando 19 textos completos.
-> Gráfico 2D Mejorado generado. Mostrando HTML...


(Pasa el ratón sobre los puntos para ver detalles)

--- Fin de Celda 4 - 2D con Conteos) ---


In [128]:
# @title 5: Visualización para Segmentos 2D/3D

import plotly.express as px
from IPython.display import display, HTML, clear_output
import pandas as pd
import numpy as np
import textwrap
import re

print("--- Celda 5: Diagnóstico Segmentos ---")

# --- 1. Verificar Datos de Entrada ---
print("\n--- 1. Verificando DataFrames de Entrada ---")
variables_ok_viz_seg = True
df_segmentos_viz = globals().get('df_segmentos_viz')
df_palabras_viz = globals().get('df_palabras_viz')

if df_segmentos_viz is None or not isinstance(df_segmentos_viz, pd.DataFrame):
    print("(!) Error: DataFrame 'df_segmentos_viz' NO encontrado o inválido.")
    variables_ok_viz_seg = False
elif df_segmentos_viz.empty:
    print("(!) Error: DataFrame 'df_segmentos_viz' ESTÁ VACÍO.")
    variables_ok_viz_seg = False
else:
    print(f"-> df_segmentos_viz: OK ({len(df_segmentos_viz)} filas). Columnas: {df_segmentos_viz.columns.tolist()}")

if df_palabras_viz is None or not isinstance(df_palabras_viz, pd.DataFrame):
    print("(!) Advertencia: 'df_palabras_viz' no encontrado. Conteo de palabras no se calculará.")
    # No marcamos como error fatal aquí, solo no habrá conteo
elif df_palabras_viz.empty:
    print("(!) Advertencia: 'df_palabras_viz' está vacío. Conteo de palabras será 0.")


# --- 2. Calcular Conteos y Preparar Hover (Si datos base están OK) ---
print("\n--- 2. Calculando Conteos y Preparando Hover ---")
if variables_ok_viz_seg:
    # Eliminar columnas si existen para evitar MergeError
    cols_to_drop = ['segment_count', 'word_count_seg', 'texto_snippet', 'hover_text_display', 'texto_length', 'segmento_texto_temp', 'segmento_texto_lookup'] # Añadir todas las posibles
    cols_dropped = []
    for col in cols_to_drop:
        if col in df_segmentos_viz.columns:
            df_segmentos_viz = df_segmentos_viz.drop(columns=col); cols_dropped.append(col)
    if cols_dropped: print(f"  -> Columnas pre-existentes eliminadas: {cols_dropped}")

    # Conteo de Palabras por Segmento
    if df_palabras_viz is not None and not df_palabras_viz.empty and all(c in df_palabras_viz.columns for c in ['texto_id', 'segmento_idx']):
         try:
             word_counts_per_segment = df_palabras_viz.groupby(['texto_id', 'segmento_idx'], observed=False).size().reset_index(name='word_count_seg') # observed=False previene warnings
             df_segmentos_viz = pd.merge(df_segmentos_viz, word_counts_per_segment, on=['texto_id', 'segmento_idx'], how='left')
             df_segmentos_viz['word_count_seg'] = df_segmentos_viz['word_count_seg'].fillna(0).astype(int)
             print("  -> Conteo de palabras por segmento añadido/actualizado.")
         except Exception as e_count:
             print(f"  (!) Error calculando/fusionando conteo de palabras: {e_count}")
             if 'word_count_seg' not in df_segmentos_viz.columns: df_segmentos_viz['word_count_seg'] = 0
    elif 'word_count_seg' not in df_segmentos_viz.columns:
        df_segmentos_viz['word_count_seg'] = 0; print("  (!) Conteo de palabras por segmento no calculado (faltan datos).")

    # Preparar Hover Text (Usando texto del JSON si está, sino fallback)
    print("  Preparando columna 'hover_text'...")
    if 'analisis_json_completo' in df_segmentos_viz.columns: # Intentar usar el JSON dentro del DF de segmentos
        def get_seg_text_from_json_col(row):
             json_list = row.get('analisis_json_completo')
             if isinstance(json_list, list) and json_list:
                  # Reconstruir texto uniendo las palabras
                  words = [str(tok.get('palabra_analizada', tok.get('texto', ''))) for tok in json_list]
                  full_seg_text = " ".join(words)
                  # Eliminar espacios extra causados por puntuación, etc. (simplificación)
                  full_seg_text = re.sub(r'\s([,.!?;:])', r'\1', full_seg_text)
                  return full_seg_text
             elif pd.notna(row.get('segmento_texto')): # Usar columna si existe
                  return row['segmento_texto']
             else: # Fallback
                  return f"ID:{row.get('texto_id','?')}/Seg:{row.get('segmento_idx','?')}"

        df_segmentos_viz['segmento_texto_temp'] = df_segmentos_viz.apply(get_seg_text_from_json_col, axis=1)
    elif 'segmento_texto' in df_segmentos_viz.columns: # Si no hay JSON pero sí texto
         df_segmentos_viz['segmento_texto_temp'] = df_segmentos_viz['segmento_texto']
    else: # Si no hay nada
         df_segmentos_viz['segmento_texto_temp'] = "Texto no disponible"

    def format_hover_text_simple_seg(text, width=80):
         if not isinstance(text, str): return "N/A"
         clean = ' '.join(text.split()); wrapped = textwrap.fill(clean, width=width); return wrapped.replace('\n', '<br>')

    df_segmentos_viz['hover_text'] = df_segmentos_viz['segmento_texto_temp'].apply(format_hover_text_simple_seg)
    if 'segmento_texto_temp' in df_segmentos_viz.columns: df_segmentos_viz = df_segmentos_viz.drop(columns=['segmento_texto_temp'])
    print("  -> Columna 'hover_text' creada/actualizada.")
    # Verificar si hover_text tiene NaNs
    if df_segmentos_viz['hover_text'].isnull().any():
         print("  (!) Advertencia: Se encontraron NaNs en 'hover_text'.")
         df_segmentos_viz['hover_text'] = df_segmentos_viz['hover_text'].fillna("Error Hover")


# --- 3. Parámetros de Visualización (Leídos desde @param) ---
print("\n--- 3. Configuración Leída ---")
# @markdown Método y Dimensiones:
param_metodo_seg_23d = "t-SNE" # @param ["UMAP", "PCA", "t-SNE"]
param_dimensiones_seg_23d = 3 # @param [2, 3]
# @markdown Color, Símbolo, Tamaño:
param_color_var_seg = "texto_id" # @param ["texto_id", "ninguno"]
param_symbol_var_seg = "ninguno" # @param ["texto_id", "ninguno"]
param_size_var_seg = "word_count_seg" # @param ["fijo", "word_count_seg"]
param_tamano_base_seg = 9
# @markdown Otros:
param_opacidad_seg = 0.75
param_mostrar_ejes_seg = False
param_template_seg = "plotly_white"

# Leer parámetros en variables locales
method = param_metodo_seg_23d
try:
    # --- CAMBIO AQUÍ: Añadir casting explícito (más seguro) ---
    dimensions = int(param_dimensiones_seg_23d)
    if dimensions not in [2, 3]:
        raise ValueError("Dimension debe ser 2 o 3")
    # --- FIN CAMBIO ---
except ValueError:
    print(f"(!) Error: Valor inválido para dimensiones '{param_dimensiones_seg_23d}'. Usando 2D por defecto.")
    dimensions = 2 # Default seguro
color_col = None if param_color_var_seg == "ninguno" else param_color_var_seg
symbol_col = None if param_symbol_var_seg == "ninguno" else param_symbol_var_seg
size_col = None if param_size_var_seg == "fijo" else param_size_var_seg
base_marker_size = param_tamano_base_seg
opacity = param_opacidad_seg
show_axes = param_mostrar_ejes_seg
template = param_template_seg
print(f"Config: Método={method}, Dims={dimensions}, Color={color_col}, Symbol={symbol_col}, Size={size_col}, BaseSize={base_marker_size}") # dimensions


# --- 4. Preparar Datos para Plotly ---
print("\n--- 4. Preparando Datos para Plotly ---")
df_plot_valid_seg = None # Inicializar

if not variables_ok_viz_seg:
    print("(!) Saltando preparación (datos base no OK).")
else:
    df_plot_seg = df_segmentos_viz.copy()
    # Determinar columnas de coordenadas
    x_col = f'{method.lower().replace("-","")}_{dimensions}d_x'
    y_col = f'{method.lower().replace("-","")}_{dimensions}d_y'
    z_col = f'{method.lower().replace("-","")}_{dimensions}d_z' if dimensions == 3 else None
    coord_cols = [x_col, y_col] + ([z_col] if z_col else [])
    print(f"Coordenadas a usar: {coord_cols}")

    # Columnas requeridas mínimas + coordenadas + las usadas para estética
    required_cols = ['texto_id', 'segmento_idx', 'hover_text', 'word_count_seg'] \
                    + ([color_col] if color_col else []) \
                    + ([symbol_col] if symbol_col else []) \
                    + ([size_col] if size_col else []) \
                    + coord_cols
    required_cols = sorted(list(set(required_cols))) # Únicas
    print(f"Columnas requeridas totales: {required_cols}")

    missing_cols = [col for col in required_cols if col not in df_plot_seg.columns]
    if missing_cols:
        print(f"⛔ ERROR: Faltan columnas requeridas: {', '.join(missing_cols)}")
    else:
        print("-> Todas las columnas requeridas existen.")
        # Filtrar NaNs en COORDENADAS y columna de TAMAÑO (si aplica)
        cols_to_dropna = coord_cols + ([size_col] if size_col and size_col in df_plot_seg.columns else []) # Solo añadir size_col si existe
        # Asegurar que size_col sea numérico ANTES de dropna (si existe y se usa)
        if size_col and size_col in df_plot_seg.columns:
             if not pd.api.types.is_numeric_dtype(df_plot_seg[size_col]):
                  print(f"  Convirtiendo '{size_col}' a numérico...")
                  # Usar .loc para evitar SettingWithCopyWarning aquí
                  df_plot_seg.loc[:, size_col] = pd.to_numeric(df_plot_seg[size_col], errors='coerce')
             # Ahora sí, añadir a la lista para dropna si no está ya
             if size_col not in cols_to_dropna: cols_to_dropna.append(size_col)

        print(f"Filtrando NaNs en columnas: {cols_to_dropna}")
        df_plot_valid_seg = df_plot_seg.dropna(subset=cols_to_dropna).copy()
        print(f"-> Datos válidos para graficar: {len(df_plot_valid_seg)} filas.")
        print(f"---> DEBUG: Filas restantes DESPUÉS de dropna({cols_to_dropna}): {len(df_plot_valid_seg)}")
        # --- Añadir una comprobación extra ---
        if len(df_plot_valid_seg) == 0 and len(df_plot_seg) > 0:
             print(f"  (!) ADVERTENCIA: El DataFrame quedó vacío después de eliminar NaNs en {cols_to_dropna}. ¿Faltan coordenadas {dimensions}D válidas o la columna de tamaño tiene solo NaNs?")
             # Opcional: Mostrar las filas que se eliminaron
             # print("   Primeras filas con NaNs en esas columnas:")
             # display(df_plot_seg[df_plot_seg[cols_to_dropna].isnull().any(axis=1)].head())
        # --- Fin comprobación extra ---

        print(f"-> Datos válidos para graficar: {len(df_plot_valid_seg)} filas.")

# --- 5. Generación y Visualización 2D/3D ---
clear_output(wait=True) # Limpiar antes de mostrar
print(f"--- 5. Generando Gráfico {dimensions}D de Segmentos ---")

if df_plot_valid_seg is None or df_plot_valid_seg.empty:
    print("(!) No hay datos válidos para generar el gráfico.")
else:
    title = f'Mapa Semántico {dimensions}D de Segmentos ({method})'
    # Preparar Hover (Texto ID, Seg Idx, Conteos, Texto Segmento)
    custom_data_cols = ['texto_id', 'segmento_idx', 'hover_text', 'word_count_seg']
    custom_hovertemplate = (f"<b>Txt:</b> %{{customdata[0]}}|<b>Seg:</b> %{{customdata[1]}}<br>" # Más corto
                          f"<b>Palabras:</b> %{{customdata[3]}}<br><br>"
                          f"<b>Segmento:</b><br>%{{customdata[2]}}<extra></extra>")
    # Preparar mapas de color/símbolo
    color_discrete_map = None; symbol_map = None; unique_colors = []; unique_symbols = []
    if color_col: unique_colors = sorted(df_plot_valid_seg[color_col].unique()); colors = px.colors.qualitative.Plotly; color_discrete_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_colors)}
    if symbol_col: unique_symbols = sorted(df_plot_valid_seg[symbol_col].unique()); symbols = ['circle', 'square', 'diamond', 'cross', 'x']; symbol_map = {val: symbols[i % len(symbols)] for i, val in enumerate(unique_symbols)}

    try:
        print("Construyendo figura Plotly Express...")
        plot_func = px.scatter if dimensions == 2 else px.scatter_3d
        plot_args = {'data_frame': df_plot_valid_seg, 'x': x_col, 'y': y_col, 'title': title,
                     'color': color_col, 'symbol': symbol_col, 'color_discrete_map': color_discrete_map, 'symbol_map': symbol_map,
                     'size': size_col, 'custom_data': custom_data_cols, 'template': template }
        if dimensions == 3: plot_args['z'] = z_col
        if size_col: plot_args['size_max'] = base_marker_size * 2.5
        fig_seg_23d = plot_func(**plot_args)

        print("Actualizando trazas y layout...")
        marker_dict_update = {'opacity': opacity, 'line': dict(width=0.5, color='DarkSlateGrey')}
        if not size_col: marker_dict_update['size'] = base_marker_size
        fig_seg_23d.update_traces(marker=marker_dict_update, hovertemplate=custom_hovertemplate)

        legend_title_parts = []
        if color_col: legend_title_parts.append(f"Color: {color_col.replace('_',' ').title()}")
        if symbol_col: legend_title_parts.append(f"Símbolo: {symbol_col.replace('_',' ').title()}")
        layout_config = {'hovermode': 'closest', 'margin': dict(l=20, r=20, t=60, b=20),
                         'showlegend': bool(color_col or symbol_col), 'legend_title_text': "<br>".join(legend_title_parts)}
        if dimensions == 2:
             layout_config['xaxis'] = dict(visible=show_axes, title=f"{method} Dim 1" if show_axes else None, zeroline=False, showgrid=False)
             layout_config['yaxis'] = dict(visible=show_axes, title=f"{method} Dim 2" if show_axes else None, zeroline=False, showgrid=False)
        else:
             layout_config['scene'] = dict(xaxis=dict(visible=show_axes, title=f"{method} Dim 1" if show_axes else ''),
                                           yaxis=dict(visible=show_axes, title=f"{method} Dim 2" if show_axes else ''),
                                           zaxis=dict(visible=show_axes, title=f"{method} Dim 3" if show_axes else ''))
             if not show_axes: layout_config['scene'].update(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
        fig_seg_23d.update_layout(**layout_config)

        # Mostrar como HTML
        print("-> Gráfico generado. Mostrando HTML...")
        html_seg_23d = fig_seg_23d.to_html(full_html=False, include_plotlyjs='cdn')
        display(HTML(html_seg_23d))
        print("(Pasa el mouse sobre los puntos para ver detalles del segmento)")

    except Exception as e:
        print(f"⛔ ERROR al generar/mostrar el gráfico {dimensions}D de segmentos: {e}"); import traceback; traceback.print_exc()

print(f"\n--- Fin de Celda 5  - {param_dimensiones_seg_23d}D Segmentos ---")

--- 5. Generando Gráfico 3D de Segmentos ---
Construyendo figura Plotly Express...
Actualizando trazas y layout...
-> Gráfico generado. Mostrando HTML...


(Pasa el mouse sobre los puntos para ver detalles del segmento)

--- Fin de Celda 5  - 3D Segmentos ---


In [129]:
# @title 6.A: Inspeccionar Contenido de Columnas JSON Sospechosas

import pandas as pd
import numpy as np
from IPython.display import display, Markdown

print("--- Celda 6.A: Inspección de Columnas JSON ---")

# --- 1. Verificar DataFrame ---
print("\nVerificando DataFrame 'df_palabras_viz'...")
df_to_inspect = globals().get('df_palabras_viz') # Usar el DF original cargado

if df_to_inspect is None or not isinstance(df_to_inspect, pd.DataFrame) or df_to_inspect.empty:
    print("(!) Error: DataFrame 'df_palabras_viz' no encontrado o está vacío.")
    print("    Asegúrate de haber cargado los datos correctamente en una celda anterior.")
else:
    print(f"-> DataFrame encontrado ({len(df_to_inspect)} filas).")

    # --- 2. Definir Columnas a Inspeccionar ---
    # Columnas que sospechamos contienen listas o estructuras complejas
    cols_to_check = [
        'ejemplos_uso',
        'usos_comunes', # Otra posible lista
        # Añade aquí otras columnas que quieras verificar si sospechas de ellas
        # 'alguna_otra_columna_lista_o_array'
    ]
    print(f"\nColumnas a inspeccionar: {cols_to_check}")

    # --- 3. Iterar e Inspeccionar ---
    for col_name in cols_to_check:
        print(f"\n--- Inspeccionando Columna: '{col_name}' ---")
        if col_name not in df_to_inspect.columns:
            print("  -> Columna NO encontrada en el DataFrame.")
            continue # Saltar a la siguiente columna

        # Contar valores nulos/NaN
        null_count = df_to_inspect[col_name].isnull().sum()
        print(f"  -> Nulos/NaN: {null_count} ({null_count / len(df_to_inspect):.1%})")

        # Obtener los tipos de datos únicos presentes en la columna (ignorando NaN)
        # Usamos una función para manejar posibles errores si hay datos muy extraños
        def get_type_safely(x):
             try: return type(x)
             except: return "ErrorObteniendoTipo"
        unique_types = df_to_inspect[col_name].dropna().apply(get_type_safely).unique()
        print(f"  -> Tipos de datos encontrados (sin NaN): {unique_types}")

        # Mostrar algunos ejemplos de valores NO NULOS
        print("  -> Ejemplos de valores NO NULOS:")
        non_null_examples = df_to_inspect[df_to_inspect[col_name].notna()][col_name].head(5) # Primeros 5 no nulos

        if non_null_examples.empty:
            print("     (No se encontraron valores no nulos para mostrar)")
        else:
            for idx, value in non_null_examples.items():
                # Mostrar índice, tipo y valor (limitado si es muy largo)
                value_str = repr(value) # Usar repr para ver mejor la estructura
                if len(value_str) > 150:
                     value_str = value_str[:150] + "..."
                print(f"     - Índice {idx}: Tipo={type(value).__name__}, Valor={value_str}")

        # Verificación adicional específica para listas/arrays
        is_list_or_array = df_to_inspect[col_name].dropna().apply(lambda x: isinstance(x, (list, np.ndarray)))
        if is_list_or_array.any():
             print(f"  -> Contiene Listas/Arrays? Sí ({is_list_or_array.sum()} filas)")
             # Opcional: Ver ejemplos de longitud de estas listas/arrays
             # lengths = df_to_inspect[is_list_or_array][col_name].apply(len)
             # print(f"     Distribución de longitudes (ejemplos): {lengths.describe()}")
        else:
             print("  -> Contiene Listas/Arrays? No detectado.")

    print(f"\n--- Fin de Inspección ---")

--- Celda 6.A: Inspección de Columnas JSON ---

Verificando DataFrame 'df_palabras_viz'...
-> DataFrame encontrado (990 filas).

Columnas a inspeccionar: ['ejemplos_uso', 'usos_comunes']

--- Inspeccionando Columna: 'ejemplos_uso' ---
  -> Nulos/NaN: 93 (9.4%)
  -> Tipos de datos encontrados (sin NaN): [<class 'list'>]
  -> Ejemplos de valores NO NULOS:
     - Índice 0: Tipo=list, Valor=['Me duele la cabeza.', 'Me compraron un regalo.']
     - Índice 1: Tipo=list, Valor=['Me gustaría ir al cine.', '¿Te gustaría un café?']
     - Índice 2: Tipo=list, Valor=['Necesito cortar el césped.', 'Voy a cortar el pastel.']
     - Índice 3: Tipo=list, Valor=['El libro está sobre la mesa.', 'El perro ladra mucho.']
     - Índice 4: Tipo=list, Valor=['Tiene el pelo rubio.', 'Se peina el pelo todas las mañanas.']
  -> Contiene Listas/Arrays? Sí (897 filas)

--- Inspeccionando Columna: 'usos_comunes' ---
  -> Nulos/NaN: 873 (88.2%)
  -> Tipos de datos encontrados (sin NaN): [<class 'list'>]
  -> Ejemp

In [133]:
# @title 6 (v10): Visualización 2D/3D PALABRAS (Filtrado por Texto ID + Líneas Opcionales)

import plotly.express as px
import plotly.graph_objects as go # Necesario para líneas 3D
from IPython.display import display, HTML, clear_output, Markdown
import pandas as pd
import numpy as np
import textwrap
import re

print("--- Celda 6 (v10): Visualización Palabras Filtradas + Líneas ---")

# --- 1. Parámetro para Seleccionar Texto ID ---
# @markdown Introduce el **texto_id EXACTO** que quieres visualizar (ej. Texto_1). Consulta la lista generada abajo.
param_texto_id_seleccionado = "Texto_18" # @param {type:"string"}

# --- 2. Verificar Datos de Entrada y GENERAR LISTA GUÍA ---
print("\n--- 2. Verificando Datos y Generando Lista Guía ---")
variables_ok_viz_words = True
df_palabras_viz = globals().get('df_palabras_viz')
dict_cot_global_viz = globals().get('dict_cot_global_viz')

if df_palabras_viz is None or not isinstance(df_palabras_viz, pd.DataFrame) or df_palabras_viz.empty:
    print("(!) Error: DataFrame 'df_palabras_viz' NO encontrado o inválido.")
    variables_ok_viz_words = False
else:
    print(f"-> df_palabras_viz: OK ({len(df_palabras_viz)} filas totales).")
    required_word_cols = ['texto_id', 'segmento_idx', 'palabra_idx_in_segment', 'palabra_texto', 'categoria'] + ['pca_2d_x']
    missing_req_word_cols = [c for c in required_word_cols if c not in df_palabras_viz.columns]
    if missing_req_word_cols:
        print(f" (!) Error: Faltan columnas esenciales: {missing_req_word_cols}")
        variables_ok_viz_words = False

if dict_cot_global_viz is None: dict_cot_global_viz = {}; print("(!) Adv: 'dict_cot_global_viz' no encontrado.")

# Generar y mostrar lista guía
unique_text_ids_global = [] # Guardar IDs para validación
if variables_ok_viz_words:
    available_texts_with_preview = []
    unique_text_ids_global = sorted(df_palabras_viz['texto_id'].unique())
    for text_id in unique_text_ids_global:
        original_text = dict_cot_global_viz.get(text_id, {}).get('texto_original', '')
        preview = textwrap.shorten(original_text.replace('\n', ' '), width=60, placeholder="...") if original_text else "(Sin preview)"
        available_texts_with_preview.append(f"`{text_id}`: {preview}")
    display(Markdown("**Textos Disponibles (Introduce el ID en el campo de arriba):**\n\n" + "\n".join([f"- {item}" for item in available_texts_with_preview])))
    # Validar selección
    if param_texto_id_seleccionado not in unique_text_ids_global:
        print(f"\n⛔ ERROR: El texto_id introducido '{param_texto_id_seleccionado}' NO existe.")
        variables_ok_viz_words = False

# --- 3. Filtrar, Preparar Hover, Crear ID Único y Ordenar ---
print(f"\n--- 3. Procesando Datos para '{param_texto_id_seleccionado}' ---")
df_palabras_procesadas = None # Cambiado el nombre para claridad
if variables_ok_viz_words:
    # Filtrar
    df_palabras_procesadas = df_palabras_viz[df_palabras_viz['texto_id'] == param_texto_id_seleccionado].copy()
    if df_palabras_procesadas.empty:
         print(f"(!) No se encontraron palabras para '{param_texto_id_seleccionado}'.")
         variables_ok_viz_words = False
    else:
         print(f"  -> Filtradas {len(df_palabras_procesadas)} palabras.")
         # Preparar Hover Text (igual que v8/v7)
         if 'hover_text' in df_palabras_procesadas.columns: df_palabras_procesadas = df_palabras_procesadas.drop(columns='hover_text')
         exclude_cols = ['texto_id', 'segmento_idx', 'palabra_idx_in_segment', 'palabra_texto','categoria', 'hover_text', 'embedding', 'error'] + [col for col in df_palabras_procesadas.columns if '_x' in col or '_y' in col or '_z' in col]
         json_detail_cols = [col for col in df_palabras_procesadas.columns if col not in exclude_cols]
         def format_word_hover_detailed_v4(row): # Renombrada
             word = row.get('palabra_texto', 'N/A'); cat = row.get('categoria', 'N/A'); tid = row.get('texto_id', '?'); sid = row.get('segmento_idx', '?'); pid = row.get('palabra_idx_in_segment', '?')
             hover_parts = [f"<b>{word}</b> ({cat})", f"<span style='font-size: smaller;'>ID:{tid}|Seg:{sid}|Pos:{pid}</span>", "<br style='margin: 2px 0;'>"]
             details_added = False
             for col in json_detail_cols:
                 value = row.get(col); display_val_str = None
                 try:
                     if value is None: pass
                     elif isinstance(value, (list, np.ndarray)):
                         if len(value) > 0: safe_strs = [str(item).strip() for item in value if str(item).strip()];
                         if safe_strs: display_val_str = ", ".join(safe_strs)
                     elif isinstance(value, bool):
                         if value: display_val_str = "Sí"
                     elif pd.notna(value) and str(value).strip() != '': display_val_str = str(value).strip()
                 except Exception: display_val_str = "[Error Proc.]"
                 if display_val_str:
                     if len(display_val_str) > 100: display_val_str = textwrap.shorten(display_val_str, width=100, placeholder="...")
                     hover_parts.append(f"<i>{col.replace('_',' ').title()}:</i> {display_val_str}"); details_added = True
             if not details_added: hover_parts.append("<i>(Sin detalles adicionales)</i>")
             return "<br>".join(hover_parts)
         print("  Generando 'hover_text'...")
         df_palabras_procesadas['hover_text'] = df_palabras_procesadas.apply(format_word_hover_detailed_v4, axis=1)
         df_palabras_procesadas['hover_text'] = df_palabras_procesadas['hover_text'].fillna("Error Hover")

         # Crear ID Único de Segmento (Necesario para line_group/loop)
         print("  Generando 'segmento_id_unico'...")
         if 'segmento_id_unico' in df_palabras_procesadas.columns: df_palabras_procesadas = df_palabras_procesadas.drop(columns='segmento_id_unico')
         df_palabras_procesadas['segmento_id_unico'] = df_palabras_procesadas['texto_id'].astype(str) + '_' + df_palabras_procesadas['segmento_idx'].astype(str)

         # Ordenar el DataFrame (CRUCIAL para líneas)
         print("  Ordenando DataFrame por Segmento > Palabra...")
         df_palabras_procesadas = df_palabras_procesadas.sort_values(
             by=['segmento_idx', 'palabra_idx_in_segment'], # Ya está filtrado por texto_id
             ascending=[True, True]
         )
         print("  -> Datos procesados y ordenados.")
else:
    print(f"(!) Saltando procesamiento para '{param_texto_id_seleccionado}'.")


# --- 4. Parámetros de Visualización (Re-incluyendo Líneas) ---
print(f"\n--- 4. Configuración Visualización para '{param_texto_id_seleccionado}' ---")
# @markdown Método y Dimensiones:
param_metodo_word_23d = "PCA" # @param ["UMAP", "PCA", "t-SNE"]
param_dimensiones_word_23d = 3 # @param [2, 3]
# @markdown ---
# @markdown **Puntos (Palabras):**
# @markdown Color por:
param_color_var_word = "categoria" # @param ["categoria", "texto_id", "ninguno"]
# @markdown Símbolo por:
param_symbol_var_word = "ninguno" # @param ["categoria", "texto_id", "ninguno"]
# @markdown Tamaño (fijo):
param_tamano_base_word = 7 # @param {type:"slider", min:2, max:15, step:1}
# @markdown Opacidad:
param_opacidad_word = 0.75 # @param {type:"slider", min:0.1, max:1.0, step:0.1}
# @markdown ---
# @markdown **Líneas de Conexión:**
# @markdown Mostrar líneas entre palabras secuenciales?
param_mostrar_lineas = True # @param {type:"boolean"}
# @markdown Ancho de las líneas:
param_ancho_linea = 1 # @param {type:"slider", min:0.5, max:5, step:0.5}
# @markdown Color de las líneas:
param_color_linea = "Gris" # @param ["Gris", "Coincidir con Puntos"]
# @markdown ---
# @markdown **Otros:**
# @markdown Mostrar Ejes?
param_mostrar_ejes_word = True # @param {type:"boolean"}
# @markdown Plantilla de Estilo:
param_template_word = "plotly_white" # @param ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white"]

# Leer parámetros
method = param_metodo_word_23d
try: dimensions = int(param_dimensiones_word_23d)
except ValueError: dimensions = 2; print("Dimensión inválida, usando 2D.")
color_col_word = None if param_color_var_word == "ninguno" else param_color_var_word
symbol_col_word = None if param_symbol_var_word == "ninguno" else param_symbol_var_word
base_marker_size_word = param_tamano_base_word
opacity = param_opacidad_word
show_lines = param_mostrar_lineas
line_width = param_ancho_linea if show_lines else 0
line_color_option = param_color_linea if show_lines else "Gris"
show_axes = param_mostrar_ejes_word
template = param_template_word
print(f"Config: Método={method}, Dims={dimensions}, Color={color_col_word}, Symbol={symbol_col_word}, Size=Fijo({base_marker_size_word}), Líneas={show_lines}({line_width}px, {line_color_option})")


# --- 5. Preparar Datos para Plotly (DataFrame Procesado) ---
print("\n--- 5. Preparando Datos para Plotly ---")
df_plot_valid_words = None
if not variables_ok_viz_words or df_palabras_procesadas is None:
    print("(!) Saltando preparación (datos no OK o no procesados).")
elif df_palabras_procesadas.empty:
     print(f"(!) No hay palabras válidas para '{param_texto_id_seleccionado}' para graficar.")
else:
    df_plot_words = df_palabras_procesadas.copy() # Usar el DF ya filtrado, ordenado y con hover/id_unico
    x_col = f'{method.lower().replace("-","")}_{dimensions}d_x'; y_col = f'{method.lower().replace("-","")}_{dimensions}d_y'; z_col = f'{method.lower().replace("-","")}_{dimensions}d_z' if dimensions == 3 else None
    coord_cols = [col for col in [x_col, y_col, z_col] if col is not None]
    missing_coord_cols = [c for c in coord_cols if c not in df_plot_words.columns]
    if missing_coord_cols: print(f"⛔ ERROR: Coordenadas {dimensions}D para '{method}' ({missing_coord_cols}) no existen.")
    else:
         print(f"Coordenadas a usar: {coord_cols}")
         required_cols = ['texto_id', 'segmento_idx', 'palabra_idx_in_segment', 'palabra_texto', 'categoria', 'hover_text', 'segmento_id_unico'] + ([color_col_word] if color_col_word else []) + ([symbol_col_word] if symbol_col_word else []) + coord_cols
         required_cols = sorted(list(set(required_cols)))
         missing_req_cols = [col for col in required_cols if col not in df_plot_words.columns]
         if missing_req_cols: print(f"⛔ ERROR: Faltan columnas requeridas: {', '.join(missing_req_cols)}")
         else:
             print("-> Todas las columnas requeridas existen.")
             print(f"Filtrando NaNs en columnas de coordenadas: {coord_cols}")
             rows_before_na = len(df_plot_words)
             # Validar/Convertir coords (igual)
             coords_ok_types = True; # ... (mismo código de validación de tipo de coord que antes) ...
             for c in coord_cols:
                  if not pd.api.types.is_numeric_dtype(df_plot_words[c]):
                       try: df_plot_words.loc[:, c] = pd.to_numeric(df_plot_words[c], errors='coerce')
                       except: coords_ok_types = False; break
                       if df_plot_words[c].isnull().all(): coords_ok_types = False; break
             # Dropna
             if coords_ok_types:
                  df_plot_valid_words = df_plot_words.dropna(subset=coord_cols).copy()
                  rows_after_na = len(df_plot_valid_words)
                  print(f"  -> Filas antes: {rows_before_na}, Filas después: {rows_after_na} ({rows_before_na - rows_after_na} eliminadas)")
                  if rows_after_na == 0: print(f"(!) ADVERTENCIA: No quedan palabras para '{param_texto_id_seleccionado}' tras filtrar NaNs.")
                  print(f"-> Datos válidos para graficar: {len(df_plot_valid_words)} palabras.")
             else:
                  print(" (!) No se puede continuar por problemas con tipos de coordenadas.")
                  df_plot_valid_words = pd.DataFrame()


# --- 6. Generación y Visualización 2D/3D (CON LÍNEAS) ---
clear_output(wait=True)
print(f"--- 6. Generando Gráfico {dimensions}D para '{param_texto_id_seleccionado}' (Líneas: {show_lines}) ---")

# --- RE-MOSTRAR LISTA GUÍA ---
if variables_ok_viz_words and 'available_texts_with_preview' in locals():
     display(Markdown("**Textos Disponibles (Introduce el ID en el campo de arriba):**\n\n" + "\n".join([f"- {item}" for item in available_texts_with_preview])))
     print("-" * 30)

if df_plot_valid_words is None or df_plot_valid_words.empty:
    print(f"(!) No hay datos válidos para generar el gráfico para '{param_texto_id_seleccionado}'.")
else:
    title = f'Mapa Semántico {dimensions}D Palabras: {param_texto_id_seleccionado}<br><sup>({method}, Color: {param_color_var_word}, Símbolo: {param_symbol_var_word})</sup>'
    custom_data_cols = ['hover_text']
    custom_hovertemplate = "%{customdata[0]}<extra></extra>"

    color_discrete_map = None; symbol_map = None; unique_colors = []; unique_symbols = []
    if color_col_word and color_col_word in df_plot_valid_words:
         df_plot_valid_words.loc[:, color_col_word] = df_plot_valid_words[color_col_word].fillna('N/A')
         unique_colors = sorted(df_plot_valid_words[color_col_word].unique()); palette = px.colors.qualitative.Plotly; color_discrete_map = {val: palette[i % len(palette)] for i, val in enumerate(unique_colors)}
    if symbol_col_word and symbol_col_word in df_plot_valid_words:
         df_plot_valid_words.loc[:, symbol_col_word] = df_plot_valid_words[symbol_col_word].fillna('N/A')
         unique_symbols = sorted(df_plot_valid_words[symbol_col_word].unique()); symbols_list = ['circle', 'square', 'diamond', 'cross', 'x']; symbol_map = {val: symbols_list[i % len(symbols_list)] for i, val in enumerate(unique_symbols)}

    # Determinar columna para agrupar líneas (si aplica)
    line_group_col = 'segmento_id_unico' if show_lines else None

    fig_word_filtered = None # Inicializar figura
    try:
        print("Construyendo figura base Plotly Express...")
        plot_func = px.scatter if dimensions == 2 else px.scatter_3d

        # Definir args base (SIN line_group inicialmente)
        plot_args = {'data_frame': df_plot_valid_words, 'x': x_col, 'y': y_col, 'title': title,
                     'color': color_col_word, 'symbol': symbol_col_word,
                     'color_discrete_map': color_discrete_map, 'symbol_map': symbol_map,
                     'size': None, 'custom_data': custom_data_cols, 'template': template }
        if dimensions == 3: plot_args['z'] = z_col
        # Añadir line_group SOLO si es 2D y se piden líneas
        if dimensions == 2 and show_lines:
             plot_args['line_group'] = line_group_col

        # Crear figura base (puntos + líneas 2D si aplica)
        fig_word_filtered = plot_func(**plot_args)

        print("Actualizando trazas de puntos...")
        marker_dict_update = {'opacity': opacity, 'size': base_marker_size_word, 'line': dict(width=0)}
        fig_word_filtered.update_traces(marker=marker_dict_update, hovertemplate=custom_hovertemplate, selector=dict(mode='markers'))

        # --- Lógica para Añadir/Configurar Líneas ---
        if dimensions == 3 and show_lines:
            print(f"Añadiendo trazas de líneas 3D...")
            line_color_value = 'rgba(128, 128, 128, 0.6)' # Gris por defecto
            # Iterar sobre cada segmento único EN LOS DATOS VALIDADOS
            for seg_id in df_plot_valid_words['segmento_id_unico'].unique():
                segment_data = df_plot_valid_words[df_plot_valid_words['segmento_id_unico'] == seg_id]
                if len(segment_data) >= 2:
                    fig_word_filtered.add_trace(go.Scatter3d(
                        x=segment_data[x_col], y=segment_data[y_col], z=segment_data[z_col],
                        mode='lines', line=dict(color=line_color_value, width=line_width),
                        hoverinfo='none', showlegend=False
                    ))
            print(" -> Trazas de líneas 3D añadidas.")
        elif dimensions == 2 and show_lines:
             print("Actualizando traza de líneas 2D...")
             line_color_value = None # Heredar color por defecto
             if line_color_option == "Gris": line_color_value = 'rgba(128, 128, 128, 0.6)'
             # Actualizar las líneas creadas por px.scatter
             fig_word_filtered.update_traces(line=dict(color=line_color_value, width=line_width), selector=dict(mode='lines'))
             print(" -> Traza de líneas 2D actualizada.")
        # -----------------------------------------

        print("Configurando layout final...")
        # Configurar layout (igual que v8/v9)
        legend_title_parts = []
        if color_col_word: legend_title_parts.append(f"Color: {color_col_word.replace('_',' ').title()} ({len(unique_colors)})")
        if symbol_col_word: legend_title_parts.append(f"Símbolo: {symbol_col_word.replace('_',' ').title()} ({len(unique_symbols)})")
        layout_config = {'hovermode': 'closest', 'margin': dict(l=10, r=10, t=80, b=10),
                         'legend_title_text': "<br>".join(legend_title_parts),
                         'showlegend': bool(color_col_word or symbol_col_word) and (len(unique_colors) + len(unique_symbols) < 40) }
        if dimensions == 2:
             layout_config['xaxis'] = dict(visible=show_axes, title=f"{method} Dim 1" if show_axes else None, zeroline=False, showgrid=False)
             layout_config['yaxis'] = dict(visible=show_axes, title=f"{method} Dim 2" if show_axes else None, zeroline=False, showgrid=False)
        else: # 3D
             layout_config['scene'] = dict(xaxis=dict(visible=show_axes, title=f"{method} Dim 1" if show_axes else ''),
                                           yaxis=dict(visible=show_axes, title=f"{method} Dim 2" if show_axes else ''),
                                           zaxis=dict(visible=show_axes, title=f"{method} Dim 3" if show_axes else ''))
             if not show_axes: layout_config['scene'].update(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
        fig_word_filtered.update_layout(**layout_config)

        print(f"-> Gráfico para '{param_texto_id_seleccionado}' generado. Mostrando HTML...")
        html_word_filtered = fig_word_filtered.to_html(full_html=False, include_plotlyjs='cdn')
        display(HTML(html_word_filtered))
        print("(Pasa el mouse sobre los puntos para ver detalles completos de la palabra)")

    except Exception as e:
        print(f"⛔ ERROR al generar/mostrar el gráfico para '{param_texto_id_seleccionado}': {e}"); import traceback; traceback.print_exc()

print(f"\n--- Fin de Celda 6 (v10) - Visualización Filtrada + Líneas Opcionales ---")

--- 6. Generando Gráfico 3D para 'Texto_18' (Líneas: True) ---


**Textos Disponibles (Introduce el ID en el campo de arriba):**

- `Texto_1`: Me gustaría cortarle el pelo, ya está muy largo
- `Texto_10`: El libro que me recomendaste ayer tiene una trama muy...
- `Texto_11`: ¡Caramba!, no esperaba encontrarte aquí.
- `Texto_12`: Me va a costar un ojo de la cara
- `Texto_13`: pucha vecino, no me quedan más bolsitas
- `Texto_14`: Tía no hay no como que eso pepino nada tortuga y sandía...
- `Texto_15`: Tengo ganas de interactuar con la página
- `Texto_16`: Esta enfermedad de amanecer nostálgico En la oscuridad de...
- `Texto_17`: El presidente anunció nuevas medidas económicas durante...
- `Texto_18`: Ahí va el capitán Beto por el espacio Con su nave de...
- `Texto_19`: La bruma espesa, eterna, para que olvide dónde me ha...
- `Texto_2`: Necesito ir al banco.
- `Texto_3`: Él vino tarde y trajo el vino que prometió.
- `Texto_4`: La copia del informe está lista; el estudiante copia...
- `Texto_5`: Bajo la intensa lluvia, bajo al refugio rápidamente.
- `Texto_6`: Vi al hombre con el telescopio.
- `Texto_7`: Compramos pasteles y bebidas frías para la fiesta.
- `Texto_8`: El perro persiguió al gato hasta que se cansó.
- `Texto_9`: El sol brilla fuerte y los niños juegan alegres en el...

------------------------------
Construyendo figura base Plotly Express...
Actualizando trazas de puntos...
Añadiendo trazas de líneas 3D...
 -> Trazas de líneas 3D añadidas.
Configurando layout final...
-> Gráfico para 'Texto_18' generado. Mostrando HTML...


(Pasa el mouse sobre los puntos para ver detalles completos de la palabra)

--- Fin de Celda 6 (v10) - Visualización Filtrada + Líneas Opcionales ---
